In [1]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

In [11]:
# Инициализация модели для эмбеддингов
embeddings = HuggingFaceEmbeddings(model_name="blanchefort/rubert-base-cased-sentiment-mokoron")

No sentence-transformers model found with name blanchefort/rubert-base-cased-sentiment-mokoron. Creating a new one with mean pooling.


# Create docs
____

In [4]:
from datasets import load_dataset

dataset = load_dataset("Den4ikAI/russian_cleared_wikipedia")
dataset

DatasetDict({
    train: Dataset({
        features: ['sample'],
        num_rows: 6284
    })
})

In [9]:
# Создаем список документов (можно заменить на загрузку из файла)
def create_docs(dataset, num_samples=6_000, seed=42):
    texts = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    documents = [
        Document(
            page_content=doc["sample"],
            metadata={"source": "wiki_ai", "page": i}
        )
        for i, doc in enumerate(texts)
    ]
    return documents

In [10]:
documents = create_docs(dataset=dataset)

In [12]:
# Создаем или подключаемся к коллекции в Qdrant
qdrant_db = Qdrant.from_documents(
    documents=documents,
    embedding=embeddings,
    url="http://localhost:6333",  # Адрес Qdrant
    collection_name="ai_documents",
    force_recreate=True  # Пересоздать коллекцию если существует
)